In [ ]:
from tkinter import *
from tkinter.ttk import *
from tkinter import scrolledtext 
from tkinter import ttk
import tk_tools as tkt
import tkinter as tk
import paho.mqtt.client as mqtt
import serial.tools.list_ports
from random import randint
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Connect to ohstem to get values from sensors
MQTT_SERVER = "mqtt.ohstem.vn"                                                                                                                                                                                      
MQTT_PORT = 1883                                                                                                                                                                                                                                                                                                                                  
MQTT_USERNAME = "group114"                                                                                                                                                                    
MQTT_PASSWORD = ""                                                                                                                                
MQTT_TOPIC_PUB_TEMP = MQTT_USERNAME + "/feeds/temp/V1"                                                                                                                                  
MQTT_TOPIC_PUB_MOI = MQTT_USERNAME + "/feeds/moi/V1"                                                                                                                        
MQTT_TOPIC_SUB_BUTTON_LIGHT = MQTT_USERNAME + "/feeds/light/V2"
MQTT_TOPIC_PUB_BUTTON_LIGHT = MQTT_USERNAME + "/feeds/light/V1"
MQTT_TOPIC_SUB_BUTTON_PUMP = MQTT_USERNAME + "/feeds/pump/V2"
MQTT_TOPIC_PUB_BUTTON_PUMP = MQTT_USERNAME + "/feeds/pump/V1"
MQTT_TOPIC_SUB_BUTTON_FAN = MQTT_USERNAME + "/feeds/fan/V2"
MQTT_TOPIC_PUB_BUTTON_FAN = MQTT_USERNAME + "/feeds/fan/V1"

def mqtt_connected(client, userdata, flags, rc):
    print("Connected succesfully!!")
    client.subscribe(MQTT_TOPIC_PUB_TEMP)
    client.subscribe(MQTT_TOPIC_PUB_MOI)
    client.subscribe(MQTT_TOPIC_PUB_BUTTON_LIGHT)
    client.subscribe(MQTT_TOPIC_PUB_BUTTON_FAN)
    client.subscribe(MQTT_TOPIC_PUB_BUTTON_PUMP)

def mqtt_subscribed(client, userdata, mid, granted_qos):
    print("Subscribed to Topic!!!")
    #text_area.insert(tk.END,"Subscribed to Topic!!!\n")

def mqtt_recv_message(client, userdata, message):    
    tmp = message.payload.decode("utf-8")
    print("Topic:",message.topic,"Received: ", tmp)
    # get temp from sensor
    if message.topic == MQTT_TOPIC_PUB_TEMP:
        print("T -",tmp)
        thermal_var = float(tmp)
        thermals.append(thermal_var)
        thermal_meter.set_value(thermal_var)
        plot_histogram(thermals, cb_var[0])
    # get moisture from senor
    if message.topic == MQTT_TOPIC_PUB_MOI:
        print("M -",tmp)
        moisture_var = float(tmp)
        moisures.append(moisture_var)
        moisture_meter.set_value(moisture_var)
        plot_histogram(moisures, cb_var[1])
    if message.topic == MQTT_TOPIC_PUB_BUTTON_FAN:
        print('F-',int(tmp))
        if int(tmp) == 255:
            relay1_btn.config(text="ON")
        else:
            relay1_btn.config(text="OFF")
    if message.topic == MQTT_TOPIC_PUB_BUTTON_LIGHT:
        print('L-',int(tmp))
        if int(tmp) == 255:
            relay2_btn.config(text="ON")
        else:
            relay2_btn.config(text="OFF")
    if message.topic == MQTT_TOPIC_PUB_BUTTON_PUMP:
        print('P-',int(tmp))
        if int(tmp) == 255:
            relay3_btn.config(text="ON")
        else:
            relay3_btn.config(text="OFF")

def on_combobox_select(event):
    # Get the current value of the combobox
    current_value = cr_cb.get()
    print(f'Selected: {current_value}')
    if current_value == cb_var[0]:
        plot_histogram(thermals, cb_var[0])
    if current_value == cb_var[1]:
        plot_histogram(moisures, cb_var[1])
    if current_value == cb_var[2]:
        plot_histogram(lights, cb_var[2])

def plot_histogram(data, lbl):
    if lbl == 'Thermal':
        ax.hist(thermals, bins='auto')
        ax.set_xlabel('Thermal')
        ax.set_ylabel('Frequency')
        ax.set_title('Thermal Distribution')
    if lbl == 'Moisuer':
        ax.hist(moisures, bins='auto')
        ax.set_xlabel('Moisuer')
        ax.set_ylabel('Frequency')
        ax.set_title('Moisuer Distribution')
    if lbl == 'Light':
        ax.hist(moisures, bins='auto')
        ax.set_xlabel('Light')
        ax.set_ylabel('Frequency')
        ax.set_title('Light Distribution')
    canvas.draw()

# Create GUI
root = tk.Tk()
root.title('Project IoT')
root.geometry('600x800')

# Vertical progress bars with labels
thermals = []
thermal_var = tk.DoubleVar()
thermal_lbl = ttk.Label(root, text="Thermal")
thermal_meter = tkt.RotaryScale(root, max_value=100.0, unit=' °C')

moisures = []
moisture_var = tk.DoubleVar()
moisture_lbl = ttk.Label(root, text="Moisture")
moisture_meter = tkt.RotaryScale(root, max_value=100.0, unit=' %')

lights = []
light_var = tk.DoubleVar()
light_lbl = ttk.Label(root, text="Light")
light_meter = tkt.RotaryScale(root, max_value=100.0, unit=' LUX')

relay1_btn = ttk.Button(root, text='OFF')
relay1_lbl = ttk.Label(root,text='Relay FAN')

relay2_btn = ttk.Button(root, text='OFF')
relay2_lbl = ttk.Label(root,text='Relay LIGHT')

relay3_btn = ttk.Button(root, text='OFF')
relay3_lbl = ttk.Label(root,text='Relay PUMP')

cb_var = ['Thermal', 'Moisuer', 'Light']
cr_cb = tk.StringVar()
hist_cb = ttk.Combobox(root, textvariable=cr_cb)
hist_cb['value'] = cb_var
hist_cb['state'] = 'readonly'
hist_cb.bind('<<ComboboxSelected>>', on_combobox_select)
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=root)

# Grid layout
thermal_lbl.grid(row=0, column=0, padx=10, pady=10)
thermal_meter.grid(row=1, column=0, padx=10, pady=10)

moisture_lbl.grid(row=0, column=1, padx=10, pady=10)
moisture_meter.grid(row=1, column=1, padx=10, pady=10)

light_lbl.grid(row=0, column=2, padx=10, pady=10)
light_meter.grid(row=1, column=2, padx=10, pady=10)

relay1_btn.grid(row=2, column=0, padx=10, pady=10)
relay1_lbl.grid(row=3, column=0, padx=10, pady=10)

relay2_btn.grid(row=2, column=1, padx=10, pady=10)
relay2_lbl.grid(row=3, column=1, padx=10, pady=10)

relay3_btn.grid(row=2, column=2, padx=10, pady=10)
relay3_lbl.grid(row=3, column=2, padx=10, pady=10)

hist_cb.grid(row=4, column=0, columnspan=3, padx=10, pady=10)
canvas.get_tk_widget().grid(row=5, column=0, columnspan=3, padx=10, pady=10)

# Connect to ohsteam
mqttClient = mqtt.Client()                                                                                                                                                            
mqttClient.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)                                                                                                                              
mqttClient.connect(MQTT_SERVER, int(MQTT_PORT), 60)                                                                                                                                   
#Register mqtt events
mqttClient.on_connect = mqtt_connected                                                                                                                                                
mqttClient.on_subscribe = mqtt_subscribed                                                                                                                                             
mqttClient.on_message = mqtt_recv_message
mqttClient.loop_start()
root.mainloop()

while True:
    canvas.draw()
    time.sleep(5)